In [1]:
# Import modules
import cv2
import numpy as np
import RPi.GPIO as GPIO

In [ ]:
# Stepper motor ULN2003 IN1-IN4
IN1 = 17
IN2 = 18
IN3 = 27
IN4 = 22

# Stepper motor steps for opening shield
steps_open = 128

# Stepper motor steps for closing shield
steps_close = -128

In [ ]:
# Stepper motor half-step sequence
SEQ = [
    [1,0,0,0],
    [1,1,0,0],
    [0,1,0,0],
    [0,1,1,0],
    [0,0,1,0],
    [0,0,1,1],
    [0,0,0,1],
    [1,0,0,1]
]

# Streaks
fog_streak = 0
sun_streak = 0

In [2]:
def detect_fog(image):
    """Returns true if fog detected otherwise false"""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    print(f"Laplacian Variance: {laplacian_var}")
    if laplacian_var < 50:  # Tunable threshold
        return True
    return False

In [3]:
def detect_sunlight(image):
    """Returns true if sunlight detected otherwise false"""
    height = image.shape[0]
    top_region = image[0:int(height * 0.3), :, :]
    hsv = cv2.cvtColor(top_region, cv2.COLOR_BGR2HSV)
    brightness = np.mean(hsv[:, :, 2])
    print(f"Top Brightness: {brightness}")
    if brightness > 200:  # Tunable threshold
        return True
    return False

In [ ]:
def step_motor(steps, delay=0.002):
    """Move motor by number of steps"""
    if steps > 0:
        seq = SEQ
    else:
        seq = SEQ[::-1]
        steps = -steps
    for _ in range(steps):
        for pattern in seq:
            GPIO.output(IN1, pattern[0])
            GPIO.output(IN2, pattern[1])
            GPIO.output(IN3, pattern[2])
            GPIO.output(IN4, pattern[3])
            time.sleep(delay)

In [ ]:
def cleanup_motor():
    """Turn off all motor coils"""
    GPIO.output(IN1, 0)
    GPIO.output(IN2, 0)
    GPIO.output(IN3, 0)
    GPIO.output(IN4, 0)

In [4]:
def main(fog_streak, sun_streak):
    cap = cv2.VideoCapture(0)  # 0 is default camera

    if not cap.isOpened():
        print("Camera not accessible")
        return 0
    print("Starting...")
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            fog = detect_fog(frame)
            sunlight = detect_sunlight(frame)
    
            label = ""
            if fog:
                label = "Fog Detected"
                if fog_streak >= 1:
                    sun_streak = 0
                    step_motor(steps_open)
                fog_streak += 1
            if sunlight:
                label = "Sunlight Detected"
                if sun_streak >= 1:
                    fog_streak = 0
                    step_motor(steps_close)
                sun_streak += 1
            if not sunlight and not fog:
                label = ""
                fog_streak = 0
                sun_streak = 0
    
            cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                        1, (0, 0, 255), 2, cv2.LINE_AA)
    
            cv2.imshow('Detection', frame)
    
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    except KeyboardInterrupt:
        print("Exiting...")
    finally:
        cleanup_motor()
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
if __name__ == '__main__':
    main(fog_streak, sun_streak)